<a href="https://colab.research.google.com/github/lbdagnachew/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# deliverable 2 

In [10]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.3'
spark_version = 'spark-3.1.3' 
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [26]:
import pandas as pd
import os
import numpy as np
from google.colab import files 
import io

data = files.upload()

Saving vine.csv to vine.csv


In [31]:
df = pd.read_csv(io.StringIO(data['vine.csv'].decode('utf-8')))


In [33]:
# read in vine.csv as a dataframe
df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,RTIS3L2M1F5SM,5,0,0,N,Y
1,R1ZV7R40OLHKD,5,0,0,N,Y
2,R3BH071QLH8QMC,1,0,1,N,Y
3,R127K9NTSXA2YH,3,0,0,N,Y
4,R32ZWUXDJPW27Q,4,0,0,N,Y


In [38]:
# filter to only get rows where there are at least 20 total votes
df_filtered_high_votes = df[df["total_votes"] >= 20]
df_filtered_high_votes.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
289,R3EZ0EPYLDA34S,1,14,31,N,Y
483,R2FJ94555FZH32,2,55,60,N,N


In [39]:
# rows with total_votes is at least .50
total_votes_filter1 = df_filtered_high_votes["total_votes"]
helpful_votes_filter1 = df_filtered_high_votes["helpful_votes"]
df_filter2_helpful = df_filtered_high_votes[(helpful_votes_filter1/total_votes_filter1) >= .50]
df_filter2_helpful.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
483,R2FJ94555FZH32,2,55,60,N,N
537,R1U3AR67RE273L,1,51,65,N,Y


In [40]:
# rows that are part of "vine" program
df_filter3_vine_yes = df_filter2_helpful[df_filter2_helpful["vine"]=="Y"]
df_filter3_vine_yes.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
32611,R3KKUSGFZWSUIY,5,56,63,Y,N
33112,R10FO5UKKVZBK2,3,23,23,Y,N
69680,RM4KSGEOR7MU1,5,19,24,Y,N
155361,RG7VRMYLEXD23,4,22,26,Y,N
239327,R11O4YSCPSNL6L,3,20,26,Y,N


In [41]:
# rows that are NOT part of "vine" program
df_filter4_vine_no = df_filter2_helpful[df_filter2_helpful["vine"]=="N"]
df_filter4_vine_no.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
483,R2FJ94555FZH32,2,55,60,N,N
537,R1U3AR67RE273L,1,51,65,N,Y


In [44]:
# number of 5-star paid reviews (vine=yes)
paid_five_star_reviews_num = len(df_filter3_vine_yes[df_filter3_vine_yes["star_rating"]==5])
paid_five_star_reviews_num

26

In [47]:
# total number of paid reviews (vine=yes)
total_paid_reviews = len(df_filter3_vine_yes)
total_paid_reviews

44

In [46]:
# total number of unpaid reviews (vine=no)
total_unpaid_reviews = len(df_filter4_vine_no)
total_unpaid_reviews

8661

In [45]:
# number of 5-star unpaid reviews (vine=no)
unpaid_five_star_reviews_num = len(df_filter4_vine_no[df_filter4_vine_no["star_rating"]==5])
unpaid_five_star_reviews_num

3233

In [48]:
# percent of 5-star reviews for paid (vine=yes)
percent_of_paid_5star = (paid_five_star_reviews_num/total_paid_reviews)*100
percent_of_paid_5star

59.09090909090909

In [49]:
# percent of 5-star reviews for unpaid (vine=no)
percent_of_unpaid_5star = (unpaid_five_star_reviews_num/total_unpaid_reviews)*100
percent_of_unpaid_5star

37.32825308855791